In [ ]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Ladataan data CSV-tiedostosta
file_path = "data_from_mysql_where_g160.csv"
data = pd.read_csv(file_path)

# Suodatetaan pois kaikki rivit, joissa 'sensorvalue_d' on 0
data_filtered = data[data['sensorvalue_d'] != 0]

x_data = data_filtered[['sensorvalue_a', 'sensorvalue_b', 'sensorvalue_c']].values  # x, y, z
y_data = data_filtered['sensorvalue_d'].values # suunta

# Lasketaan luokkien esiintymät
class_counts = data_filtered['sensorvalue_d'].value_counts()
print("Luokkien esiintymät:\n", class_counts)

# Skaalaus [0, 1] väliin ilman NumPy:n vektorilaskentaa
def scale_to_unit_interval(data):
    min_vals = [float('inf')] * len(data[0])
    max_vals = [float('-inf')] * len(data[0])

    for row in data:
        for col_idx, value in enumerate(row):
            if value < min_vals[col_idx]:
                min_vals[col_idx] = value
            if value > max_vals[col_idx]:
                max_vals[col_idx] = value

    scaled_data = []
    for row in data:
        scaled_row = []
        for col_idx, value in enumerate(row):
            range_val = max_vals[col_idx] - min_vals[col_idx]
            if range_val == 0:
                range_val = 1  # Vältetään nollalla jakamista
            scaled_value = (value - min_vals[col_idx]) / range_val
            scaled_row.append(scaled_value)
        scaled_data.append(scaled_row)

    return scaled_data

# Skaalataan syötteet
x_data_scaled = scale_to_unit_interval(x_data)

# Suuntaa on 6 luokkaa
num_classes = 6
y_data = keras.utils.to_categorical(y_data - 1, num_classes)

# Muutetaan x_train ja x_test NumPy-taulukoiksi
x_train, x_test, y_train, y_test = train_test_split(np.array(x_data_scaled), np.array(y_data), test_size=0.2, random_state=42)

# Määritellään malli, jossa on vain yksi dense-kerros
model = keras.Sequential(
    [
        keras.Input(shape=(x_train.shape[1],)),  # Syötemuoto (x, y, z)
        layers.Dense(num_classes, activation="softmax"),  # Yksi tiheä kerros
    ]
)

model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Koulutetaan mallia
batch_size = 128
epochs = 125

optimizer = Adam(learning_rate=0.001)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[early_stopping])

# Arvioidaan malli testidatalla
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

# Tallennetaan malli (painot ja rakenne)
model.save('my_model.keras')

# Ennusteet
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

print("Classification Report:\n", classification_report(y_test_classes, y_pred_classes))
print("Confusion Matrix:\n", confusion_matrix(y_test_classes, y_pred_classes))

# Osa 2

# Haetaan mallin painot
weights = model.get_weights()

# Painot ovat lista, jossa on numpy-taulukoita
for idx, weight in enumerate(weights):
    print(f"Painojen ja biasin {idx} muoto: {weight.shape}")
    print("")

# Aktivointifunktio ReLU
def relu(x):
    if isinstance(x, list):  # Jos x on lista
        return [max(0, val) for val in x]
    return max(0, x)  # Jos x on yksittäinen arvo

# Aktivointifunktio Softmax
def softmax(z):
    exp_values = [2.718 ** i for i in z]  # Eksponenttiarvot
    total = sum(exp_values)  # Lasketaan summan eksponentit
    return [exp_value / total for exp_value in exp_values]  # Jaa eksponenttiarvot kokonaisuudella

# Syöte
for i in range(len(data_filtered)):
    # Haetaan kunkin rivin arvot
    x = data_filtered['sensorvalue_a'].values[i]
    y = data_filtered['sensorvalue_b'].values[i]
    z = data_filtered['sensorvalue_c'].values[i]

    # Muutetaan syöte oikeaan muotoon
    input_data = [x, y, z]

#print("Input data shape:", input_data.shape)
print("Input data shape:", len(input_data))
print("Weights list length:", len(weights))
# Oikeat painot ja biasit
weights_0, bias_0 = weights[0], weights[1]


# Etenee syötteestä piilokerrosten kautta ulostuloon
def forward_propagation(input_data):
    # Tiheä kerros (dense layer) 0
    z0 = []
    for k in range(len(weights_0[0])):  # Käydään läpi piilokerroksen neuronit
        z0_value = sum(input_data[i] * weights_0[i][k] for i in range(len(input_data))) + bias_0[k]
        z0.append(z0_value)
    a0 = [relu(z) for z in z0]  # Aktivointi ReLU-funktiolla

    # Ulostulokerros (softmax)
    output = softmax(a0)  # Softmax aktivointi

    return output



# Asetetaan NumPy:n tulostustapa niin, että ei käytetä tieteellistä merkintää
np.set_printoptions(precision=6, suppress=True)

# Lasketaan tulos syötteelle (x, y, z)
result = forward_propagation(input_data)
print("\nVerkon ulostulo: (forward_propagation):", result)

# Oikea syötemuoto model.predict
input_data = np.array([input_data])  # Muutetaan NumPy-taulukoksi

# Lasketaan ennuste koulutetulla mallilla
prediction = model.predict(input_data)
print("\nEnnuste (model.predict):", prediction)

# Lasketaan ero tulosten välillä
result = result  # Varmistetaan, että result on lista
prediction = prediction[0].tolist()  # Muutetaan prediction listaksi

# Lasketaan ero
difference = [abs(r - p) for r, p in zip(result, prediction)]  # Lasketaan itseisarvoero

# Tulostetaan ero desimaaleina ilman tieteellistä muotoa
print("\nEro (absoluuttinen ero result ja prediction välillä):")
for i, diff in enumerate(difference):
    print(f"Ero {i+1}: {diff:.8f}")  # Tulostetaan desimaalimuodossa, pyöristettynä 8 desimaaliin

# Keskimääräinen ero
mean_difference = sum(difference) / len(difference)
# Tulostetaan keskimääräinen ero desimaaleina ilman tieteellistä muotoa
print("\nKeskimääräinen ero: ", f"{mean_difference:.8f}")

# Tallennetaan painot ja biasit header-tiedostoon
header_file = "neuroverkonKertoimet2.h"

with open(header_file, "w") as f:
    f.write("#ifndef NEUROVERKONKERTOIMET_H\n")
    f.write("#define NEUROVERKONKERTOIMET_H\n\n")

    # Kirjoitetaan painot ja biasit jokaiselle kerrokselle
    for idx, weight in enumerate(weights):
        if len(weight.shape) == 2:  # Painot (matriisi)
            f.write(f"float weights_{idx}[{weight.shape[0]}][{weight.shape[1]}] = {{\n")
            for row in weight:
                f.write("    {" + ", ".join(map(str, row)) + "},\n")
            f.write("};\n\n")
        elif len(weight.shape) == 1:  # Bias (vektori)
            f.write(f"float biases_{idx}[{weight.shape[0]}] = {{")
            f.write(", ".join(map(str, weight)))
            f.write("};\n\n")

    f.write("#endif // NEUROVERKONKERTOIMET_H\n")

# Lataa malli
model = tf.keras.models.load_model('my_model.keras')

# Muunna TensorFlow Lite -malliksi
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Tallenna muunnos
with open('oma_malli.tflite', 'wb') as f:
    f.write(tflite_model)

Luokkien esiintymät:
 sensorvalue_d
5.0    266
6.0    242
3.0    218
2.0    217
1.0    193
4.0    189
Name: count, dtype: int64


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 6)                   │              24 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24 (96.00 B)

 Trainable params: 24 (96.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/125
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.1375 - loss: 1.8105 - val_accuracy: 0.1509 - val_loss: 1.8221
Epoch 2/125
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1423 - loss: 1.7958 - val_accuracy: 0.1509 - val_loss: 1.8138
Epoch 3/125
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1523 - loss: 1.7804 - val_accuracy: 0.1509 - val_loss: 1.8058
Epoch 4/125
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1587 - loss: 1.7739 - val_accuracy: 0.1509 - val_loss: 1.7978
Epoch 5/125
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1397 - loss: 1.7630 - val_accuracy: 0.1509 - val_loss: 1.7901
Epoch 6/125
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1693 - loss: 1.7503 - val_accuracy: 0.1509 - val_loss: 1.7825
Epoch 7/125
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1436 - loss: 1.7579 - val_accuracy: 0.1509 - val_loss: 1.7751
Epoch 8/125
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1455 - loss: 1.7501 - val_accuracy: 0.1509 - val_loss: 1.7

<ipython-input-1-ee56beb09ec9>:115: RuntimeWarning: overflow encountered in scalar power
  exp_values = [2.718 ** i for i in z]  # Eksponenttiarvot
<ipython-input-1-ee56beb09ec9>:117: RuntimeWarning: invalid value encountered in scalar divide
  return [exp_value / total for exp_value in exp_values]  # Jaa eksponenttiarvot kokonaisuudella


Saved artifact at '/tmp/tmpezlxmcww'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 6), dtype=tf.float32, name=None)
Captures:
  135414943138912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135414916719968: TensorSpec(shape=(), dtype=tf.resource, name=None)
